# Scikit-Learn

#### Linear Regression-, KNN-Model and Random Forest Regressor for Multiple Regression 


## Load packages

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant  

sns.set_theme()

## Import Dataset

In [5]:
df = pd.read_csv("car_prices.csv", on_bad_lines="skip")

## Data inspection

In [6]:
df.head(2)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,"kia motors america, inc",20500,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,"kia motors america, inc",20800,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558811 entries, 0 to 558810
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558811 non-null  int64  
 1   make          548510 non-null  object 
 2   model         548412 non-null  object 
 3   trim          548160 non-null  object 
 4   body          545616 non-null  object 
 5   transmission  493458 non-null  object 
 6   vin           558811 non-null  object 
 7   state         558811 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558717 non-null  float64
 10  color         558062 non-null  object 
 11  interior      558062 non-null  object 
 12  seller        558811 non-null  object 
 13  mmr           558811 non-null  int64  
 14  sellingprice  558811 non-null  int64  
 15  saledate      558811 non-null  object 
dtypes: float64(2), int64(3), object(11)
memory usage: 68.2+ MB


In [8]:
print(df.isnull().sum())

year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65353
vin                 0
state               0
condition       11794
odometer           94
color             749
interior          749
seller              0
mmr                 0
sellingprice        0
saledate            0
dtype: int64


## Data transformation

In [9]:
# drop column with too many missing values
df = df.drop(['transmission'], axis=1)

# drop remaining row with one missing value
df = df.dropna()

In [10]:
# Drop irrelevant features
df = df.drop(['trim', 'vin', 'mmr', 'saledate'], axis=1)

In [11]:
print(df.isnull().sum())

year            0
make            0
model           0
body            0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
sellingprice    0
dtype: int64


In [12]:
# rename columns

df = df.rename(columns={
"make" : "brand",
"body" : "type",
"odometer" : "miles"} 
    )

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533660 entries, 0 to 558810
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          533660 non-null  int64  
 1   brand         533660 non-null  object 
 2   model         533660 non-null  object 
 3   type          533660 non-null  object 
 4   state         533660 non-null  object 
 5   condition     533660 non-null  float64
 6   miles         533660 non-null  float64
 7   color         533660 non-null  object 
 8   interior      533660 non-null  object 
 9   seller        533660 non-null  object 
 10  sellingprice  533660 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 48.9+ MB


In [14]:
# transform into lowercase

df["brand"] = df["brand"].str.lower()
df["model"] = df["model"].str.lower()
df["type"] = df["type"].str.lower()

# Categorial or numeric?

* year = categorial
* brand = categorial
* model = categorial
* type = categorial
* state = categorial
* condition = categorial
* miles = numeric
* color = categorial
* interior = categorial
* seller = categorial
* ratingprice = numeric
* sellingprice = numeric
* saledate = categorial

In [15]:
# transform to categorical:

for cat in ["year", "brand", "model", "type", "state", "condition", "color", "interior", "seller"]:
    df[cat] = df[cat].astype("category")

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533660 entries, 0 to 558810
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   year          533660 non-null  category
 1   brand         533660 non-null  category
 2   model         533660 non-null  category
 3   type          533660 non-null  category
 4   state         533660 non-null  category
 5   condition     533660 non-null  category
 6   miles         533660 non-null  float64 
 7   color         533660 non-null  category
 8   interior      533660 non-null  category
 9   seller        533660 non-null  category
 10  sellingprice  533660 non-null  int64   
dtypes: category(9), float64(1), int64(1)
memory usage: 18.4 MB


In [17]:
# summary statistics for all categorical columns
df.describe(include=['category']).transpose()

,count,unique,top,freq
year,533660,26,2012,100612
brand,533660,53,ford,91908
model,533660,768,altima,19159
type,533660,45,sedan,236445
state,533660,38,fl,79626
condition,533660.0,41.0,1.9,40404.0
color,533660,20,black,106600
interior,533660,17,black,238668
seller,533660,12739,nissan-infiniti lt,19677


## Data preprocessing pipeline

In [18]:
# Modules
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [19]:
# for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
    ])

In [20]:
# for categorical features  
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

In [21]:
# Pipeline
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_exclude="category")),
    ('cat', categorical_transformer, selector(dtype_include="category"))
        ])

In [22]:
df.head()

,year,brand,model,type,state,condition,miles,color,interior,seller,sellingprice
0,2015,kia,sorento,suv,ca,5.0,16639.0,white,black,"kia motors america, inc",21500
1,2015,kia,sorento,suv,ca,5.0,9393.0,white,beige,"kia motors america, inc",21500
2,2014,bmw,3 series,sedan,ca,4.5,1331.0,gray,black,financial services remarketing (lease),30000
3,2015,volvo,s60,sedan,ca,4.1,14282.0,white,black,volvo na rep/world omni,27750
4,2014,bmw,6 series gran coupe,sedan,ca,4.3,2641.0,gray,black,financial services remarketing (lease),67000


## Modeling

# Multiple Regression

In [23]:
# Select features for multiple regression
features= [
 'miles',
 'brand',
 'model',
 'type',
 'condition',
 'color'
  ]
X = df[features]

X.info()
print("Missing values:",X.isnull().any(axis = 1).sum())

# Create response
y = df["sellingprice"]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533660 entries, 0 to 558810
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   miles      533660 non-null  float64 
 1   brand      533660 non-null  category
 2   model      533660 non-null  category
 3   type       533660 non-null  category
 4   condition  533660 non-null  category
 5   color      533660 non-null  category
dtypes: category(5), float64(1)
memory usage: 11.2 MB
Missing values: 0


In [24]:
from sklearn.model_selection import train_test_split

# Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
from sklearn.linear_model import LinearRegression

# Create pipeline with model
lm_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lm', LinearRegression())
                        ])

In [26]:
# show pipeline
set_config(display="diagram")
# Fit model
lm_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7feb8fedc7f0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7feb8fedc250>)])),
                ('lm', LinearRegression())])

In [27]:
y_pred = lm_pipe.predict(X_test)

In [28]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

0.7871510077904085

In [29]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred)

19820180.859093003

In [30]:
mean_squared_error(y_test, y_pred, squared=False)

4451.986170137212

In [31]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, y_pred)

2881.345892972646

In [32]:
# Create a new GDP value
X_new = pd.DataFrame({
    "year": [2015],
    "brand": "kia",
    "model": "sorento",
    "type": "suv",
    "state": "ca",
    "condition": [5.0],
    "miles": [16639.0],
    "color": "white",
    "interior": "black",
    "seller": "kia motors america, inc",
})

In [33]:
X_new

,year,brand,model,type,state,condition,miles,color,interior,seller
0,2015,kia,sorento,suv,ca,5.0,16639.0,white,black,"kia motors america, inc"


In [34]:
my_prediction = lm_pipe.predict(X_new)

In [36]:
#save knn-model 
from joblib import dump

dump(lm_pipe, "lm_model.joblib")

['lm_model.joblib']

In [37]:
df_prediction = pd.DataFrame({"pred": my_prediction})


In [38]:
df_prediction

,pred
0,21454.28274


In [39]:
sample = {
    "year": [2015],
    "brand": "kia",
    "model": "sorento",
    "type": "suv",
    "state": "ca",
    "condition": [5.0],
    "miles": [16639.0],
    "color": "white",
    "interior": "black",
    "seller": "kia motors america, inc",
}

In [40]:
# KNN
from sklearn.neighbors import KNeighborsRegressor as KNR

# Create pipeline with model
knn_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('KNN Regression', KNR(n_neighbors=5))
                        ])

In [41]:
# show pipeline
set_config(display="diagram")
# Fit model
knn_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7feb8fedc7f0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7feb8fedc250>)])),
                ('KNN Regression', KNeighborsRegressor())])

In [42]:
y_pred = knn_pipe.predict(X_test)

In [43]:
#save knn-model 
from joblib import dump

dump(knn_pipe, "knn_model.joblib")

['knn_model.joblib']

In [44]:
r2_score(y_test, y_pred)

0.840973991058399

In [45]:
mean_squared_error(y_test, y_pred)

14808264.891470972

In [46]:
mean_squared_error(y_test, y_pred, squared=False)

3848.1508405298996

In [47]:
mean_absolute_error(y_test, y_pred)

2373.666221939063

In [48]:
my_prediction = knn_pipe.predict(X_new)

In [49]:
df_prediction_knn = pd.DataFrame({"pred": my_prediction})

In [50]:
df_prediction_knn

,pred
0,19980.0


In [51]:
# RandomForest
from sklearn.ensemble import RandomForestRegressor as RFR

# Create pipeline with model
rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('Random Forest', RFR(n_estimators=50, max_depth=5))
                        ])

In [52]:
# show pipeline
set_config(display="diagram")
# Fit model
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7feb8fedc7f0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7feb8fedc250>)])),
                ('Random Forest',
                 RandomForestRegressor(max_depth=5, n_estimators=50))])

In [53]:
#save rf-model
from joblib import dump

dump(rf_pipe, "rf_model.joblib")

['rf_model.joblib']

In [54]:
y_pred = rf_pipe.predict(X_test)

In [55]:
r2_score(y_test, y_pred)

0.5423033107857674

In [56]:
mean_squared_error(y_test, y_pred)

42620033.408010565

In [57]:
mean_squared_error(y_test, y_pred, squared=False)

6528.402056247039

In [58]:
mean_absolute_error(y_test, y_pred)

4265.844310034042